In [1]:
### IMPORTS ###
import scipy.sparse
import scipy.stats
import numpy as np
import matplotlib.pyplot as pyplot
import sklearn.metrics as mtc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from pyemd import emd
import valores
import pickle


In [2]:
## FUNCIONES ##


class TFM:

    def __init__(self,path):
        self.X_tr = scipy.sparse.load_npz(path+'X_tr.npz')
        self.X_tst = scipy.sparse.load_npz(path+'X_tst.npz')
        self.y_tr = np.load(path+'y_tr.npy')
        self.y_tst = np.load(path+'y_tst.npy')
        # Se binarizan las caracteristicas 
        self.y_tr=MultiLabelBinarizer().fit_transform(self.y_tr)
        self.y_tst=MultiLabelBinarizer().fit_transform(self.y_tst)
        # Escalado 
        scaler = MaxAbsScaler().fit(self.X_tr)
        self.X_tr=scaler.transform(self.X_tr)
        self.X_tst=scaler.transform(self.X_tst)
        self.X_tr_simpli= None
        self.simplificador = False
        self.modelo = None
        self.clf= None
        self.columns= None
        print ('Dataset cargado y preprocesado')

    def reduce_features(self,num_data):
        columns=np.array([])
        for i in range(np.shape(self.X_tr)[1]):
            if self.X_tr[:,i].count_nonzero()> num_data:
                columns=np.append(columns,i)
        np.savetxt('features'+num_data+'.txt',columns,delimiter=',')
        print('Las características han sido reducidas')

    def simplify_dataset(self,features,tr_size):
        self.X_tr_simpli=self.X_tr
        self.y_tr_simpli=self.y_tr
        
        self.columns=np.loadtxt('features'+features+'.txt',delimiter=',')
        self.X_tr_simpli=self.X_tr[:,self.columns]
        self.X_tr_simpli, __, self.y_tr_simpli, __ = train_test_split(self.X_tr_simpli, self.y_tr_simpli, test_size=(1-tr_size), random_state=42)
        print('Simplificación finalizada')
        print(np.shape(self.X_tr_simpli))

    def use_simplify(self,boolean):
        self.simplificador= boolean
        
    def grid_search_cv(self,classifier,parameters,metodo):
        if (metodo == 0):
            classif = OneVsRestClassifier(classifier)
        elif (metodo==1):
            classif = ClassifierChain(classifier)
        model_tunning = GridSearchCV(classif, param_grid=parameters,cv=4,verbose=50, n_jobs=-1,scoring='accuracy')
        if (self.simplificador):
            model_tunning.fit(self.X_tr_simpli, self.y_tr_simpli)
        else :
            model_tunning.fit(self.X_tr, self.y_tr)
        print (model_tunning.best_score_)
        print (model_tunning.best_params_)
        self.modelo = model_tunning.best_estimator_

    def fitting_classifier(self):
        if (self.simplificador):
            self.clf=self.modelo.fit(self.X_tr,self.y_tr)
        else :
            self.clf=self.modelo

    def metrics(self):
        if (self.simplificador):
            self.y_pred=self.clf.predict(self.X_tst)
        else :
            self.y_pred=self.clf.predict(self.X_tst)
        accuracy=mtc.accuracy_score(self.y_tst,self.y_pred)
        hamming=mtc.hamming_loss(self.y_tst,self.y_pred)
        precision=mtc.precision_score(self.y_tst,self.y_pred,average='micro')
        print("Total accuracy: ", accuracy)
        print("Hamming loss: ", hamming)
        print("Precision: ", precision)
        print(mtc.classification_report(self.y_tst,self.y_pred))
        #print("Accuracy per class:")
        aux=0
        for i in range(np.shape(self.y_pred)[1]):
            #print ("Class " ,i,": " ,mtc.accuracy_score(self.y_tst[:,i],y_pred[:,i]))
            aux=aux+mtc.accuracy_score(self.y_tst[:,i],self.y_pred[:,i])
        print("Accuracy media: ",aux/37)
        distance_matrix=np.ones((37,37))*(1/37)
        np.fill_diagonal(distance_matrix,0)
        emd_aux=0
        for i in range (np.shape(self.y_tst)[0]):
            emd_aux= emd_aux + emd(self.y_tst[i,:].astype(float),self.y_pred[i,:].astype(float),distance_matrix)
        
        print("EMD: ", emd_aux/(np.shape(self.y_tst)[0]))
        print("METRICAS DE LABEL RANKING: ")
        spearman=scipy.stats.spearmanr(self.y_tst, self.y_pred)[0]
        print("Spearmans Rank Correlation Coefficient: ", spearman)
        kendall=scipy.stats.kendalltau(self.y_tst, self.y_pred)[0]
        print("Kendall’s tau Correlation Coefficient: ",kendall)
        
        



In [5]:
path_string=input('Introduce el directorio del dataset')
clasificacion=TFM(path_string)
print('Inicio del programa clasificador')

while True:
    
    ask2=int(input('Que clasificador vas a usar?: '))
    clasificador=valores.clasificadores_dict[ask2]
    metod=int(input('Que metodo vas a usar?: Introduce 0 para One vs Rest, o 1 para Classifier Chain'))
        
    while True:
        ask1=input(('Quieres usar simplificacion para entrenar el algoritmo?'))
        if (ask1.lower()=='si'):
            non_zero_data=input(('Introduce el numero de elementos no nulos por categoria que consideras aceptable'))
            training_size=float(input('Introduce el porcentaje de tamano de datos de entrenamiento que quieres usar para simplificar'))
            print('Realizando simplificacion del dataset...')
            clasificacion.simplify_dataset(non_zero_data,training_size)
            clasificacion.use_simplify(True)
        elif (ask1.lower()=='no'):
            clasificacion.use_simplify(False)
        print("Iniciando tunning de los parámetros:")
        clasificacion.grid_search_cv(clasificador,valores.parametros_dict[(clasificador,metod)],metod)    
        print("Tunning finalizado:")  
        ask5=input('Quieres continuar con el entrenamiento?')
        if (ask5.lower()=='no'):
            break
        
        print("Entrenando el algoritmo con los mejores parametros...")
        clasificacion.fitting_classifier()
        print("Entrenamiento finalizado, se procede a predecir las etiquetas")
        print("Mostrando metricas obtenidas")
        clasificacion.metrics()
        ask3=input('Quieres probar a variar la simplificacion?')
        if (ask3.lower()=='si'):
            continue
        elif (ask3.lower()=='no'):
            break
            
    ask4=input('Quieres probar con otro clasificador o finalizamos el programa?')
    if (ask4.lower()=='si'):
        continue
    elif (ask4.lower()=='no'):
        break

print("Fin del programa clasificador")

Introduce el directorio del dataset../../Datasets/dataset/
Dataset cargado y preprocesado
Inicio del programa clasificador
Que clasificador vas a usar?: 1
Que metodo vas a usar?: Introduce 0 para One vs Rest, o 1 para Classifier Chain0
Quieres usar simplificacion para entrenar el algoritmo?si
Introduce el numero de elementos no nulos por categoria que consideras aceptable500
Introduce el porcentaje de tamano de datos de entrenamiento que quieres usar para simplificar0.4
Realizando simplificacion del dataset...
Simplificación finalizada
(37579, 2823)
Iniciando tunning de los parámetros:
Fitting 4 folds for each of 432 candidates, totalling 1728 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1

[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 123 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 125 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 126 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 127 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 131 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  6.7min
[Paralle

[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done 257 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 258 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 259 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 260 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 261 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 262 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 263 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 265 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 267 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 269 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 271 tasks      | elapsed: 12.9min
[Paralle

[Parallel(n_jobs=-1)]: Done 389 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 390 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 391 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 392 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 393 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 394 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 395 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 396 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 397 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 398 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 399 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 401 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done 403 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done 404 tasks      | elapsed: 19.0min
[Paralle

[Parallel(n_jobs=-1)]: Done 523 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 524 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 525 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done 526 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done 527 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done 529 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done 530 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done 531 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 532 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 533 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 534 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 535 tasks      | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done 537 tasks      | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done 538 tasks      | elapsed: 25.3min
[Paralle

...

[Parallel(n_jobs=-1)]: Done 1443 tasks      | elapsed: 144.0min
[Parallel(n_jobs=-1)]: Done 1444 tasks      | elapsed: 144.0min
[Parallel(n_jobs=-1)]: Done 1445 tasks      | elapsed: 144.1min
[Parallel(n_jobs=-1)]: Done 1446 tasks      | elapsed: 144.1min
[Parallel(n_jobs=-1)]: Done 1447 tasks      | elapsed: 144.2min
[Parallel(n_jobs=-1)]: Done 1448 tasks      | elapsed: 144.2min
[Parallel(n_jobs=-1)]: Done 1449 tasks      | elapsed: 144.8min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 144.8min
[Parallel(n_jobs=-1)]: Done 1451 tasks      | elapsed: 144.8min
[Parallel(n_jobs=-1)]: Done 1452 tasks      | elapsed: 144.9min
[Parallel(n_jobs=-1)]: Done 1453 tasks      | elapsed: 145.5min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 145.5min
[Parallel(n_jobs=-1)]: Done 1455 tasks      | elapsed: 145.5min
[Parallel(n_jobs=-1)]: Done 1456 tasks      | elapsed: 145.6min
[Parallel(n_jobs=-1)]: Done 1457 tasks      | elapsed: 146.7min
[Parallel(n_jobs=-1)]: Done 1458 tasks  

[Parallel(n_jobs=-1)]: Done 1572 tasks      | elapsed: 165.2min
[Parallel(n_jobs=-1)]: Done 1573 tasks      | elapsed: 165.3min
[Parallel(n_jobs=-1)]: Done 1574 tasks      | elapsed: 165.4min
[Parallel(n_jobs=-1)]: Done 1575 tasks      | elapsed: 165.4min
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed: 165.5min
[Parallel(n_jobs=-1)]: Done 1577 tasks      | elapsed: 166.4min
[Parallel(n_jobs=-1)]: Done 1578 tasks      | elapsed: 166.4min
[Parallel(n_jobs=-1)]: Done 1579 tasks      | elapsed: 166.5min
[Parallel(n_jobs=-1)]: Done 1580 tasks      | elapsed: 166.5min
[Parallel(n_jobs=-1)]: Done 1581 tasks      | elapsed: 167.4min
[Parallel(n_jobs=-1)]: Done 1582 tasks      | elapsed: 167.5min
[Parallel(n_jobs=-1)]: Done 1583 tasks      | elapsed: 167.5min
[Parallel(n_jobs=-1)]: Done 1584 tasks      | elapsed: 167.6min
[Parallel(n_jobs=-1)]: Done 1585 tasks      | elapsed: 167.7min
[Parallel(n_jobs=-1)]: Done 1586 tasks      | elapsed: 167.7min
[Parallel(n_jobs=-1)]: Done 1587 tasks  

[Parallel(n_jobs=-1)]: Done 1701 tasks      | elapsed: 188.5min
[Parallel(n_jobs=-1)]: Done 1702 tasks      | elapsed: 188.6min
[Parallel(n_jobs=-1)]: Done 1703 tasks      | elapsed: 188.6min
[Parallel(n_jobs=-1)]: Done 1704 tasks      | elapsed: 188.7min
[Parallel(n_jobs=-1)]: Done 1705 tasks      | elapsed: 188.8min
[Parallel(n_jobs=-1)]: Done 1706 tasks      | elapsed: 188.8min
[Parallel(n_jobs=-1)]: Done 1707 tasks      | elapsed: 188.8min
[Parallel(n_jobs=-1)]: Done 1708 tasks      | elapsed: 188.9min
[Parallel(n_jobs=-1)]: Done 1709 tasks      | elapsed: 188.9min
[Parallel(n_jobs=-1)]: Done 1710 tasks      | elapsed: 189.0min
[Parallel(n_jobs=-1)]: Done 1711 tasks      | elapsed: 189.0min
[Parallel(n_jobs=-1)]: Done 1712 tasks      | elapsed: 189.0min
[Parallel(n_jobs=-1)]: Done 1713 tasks      | elapsed: 189.2min
[Parallel(n_jobs=-1)]: Done 1714 tasks      | elapsed: 189.3min
[Parallel(n_jobs=-1)]: Done 1715 tasks      | elapsed: 189.3min
[Parallel(n_jobs=-1)]: Done 1716 tasks  

/Users/mariotrapiellomendoza/tfm/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mariotrapiellomendoza/tfm/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/mariotrapiellomendoza/tfm/lib/python3.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/mariotrapiellomendoza/tfm/lib/python3.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/mariotrapiellomendoza/tfm/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarnin

Spearmans Rank Correlation Coefficient:  [[ 1.00000000e+00  2.41413609e-01 -4.03307071e-02 ... -4.01609941e-02
   6.48720488e-02 -2.43098763e-02]
 [ 2.41413609e-01  1.00000000e+00 -4.65919572e-02 ... -4.55858548e-02
   7.88162271e-02 -2.75936021e-02]
 [-4.03307071e-02 -4.65919572e-02  1.00000000e+00 ... -5.96607962e-02
  -3.12396680e-02 -2.21677706e-04]
 ...
 [-4.01609941e-02 -4.55858548e-02 -5.96607962e-02 ...  1.00000000e+00
  -1.29549592e-02 -1.49760395e-02]
 [ 6.48720488e-02  7.88162271e-02 -3.12396680e-02 ... -1.29549592e-02
   1.00000000e+00 -7.84177438e-03]
 [-2.43098763e-02 -2.75936021e-02 -2.21677706e-04 ... -1.49760395e-02
  -7.84177438e-03  1.00000000e+00]]
Kendall’s tau Correlation Coefficient:  0.6634505292549552
Quieres probar a variar la simplificacion?no
Quieres probar con otro clasificador o finalizamos el programa?no
Fin del programa clasificador


In [ ]:
path_string="../../Datasets/dataset/"
metod=0
clasificador=valores.LogisticRegression()
parameters={
    "estimator__C": [1],
    "estimator__solver": ["liblinear"],
    "estimator__tol": [0.01]
}
clasificacion=TFM(path_string)
clasificacion.use_simplify(False)
clasificacion.grid_search_cv(clasificador,parameters,metod)
clasificacion.fitting_classifier()
clasificacion.metrics()

Dataset cargado y preprocesado
Fitting 4 folds for each of 1 candidates, totalling 4 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
